# Combined Spange + DRFP (High-Variance) + Arrhenius Features

This experiment combines:
1. Spange descriptors (13 features) - physicochemical properties
2. DRFP features with non-zero variance (122 features) - molecular structure
3. Arrhenius kinetics features (5 features) - physics-informed

Total: ~140 features

Key changes from previous experiments:
- NO PCA on DRFP - use variance-based feature selection instead
- Increased regularization (dropout 0.3, weight decay 1e-4)
- Larger hidden layers [256, 128, 64] for more features

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

Using device: cuda
GPU: NVIDIA A100-SXM4-80GB


In [2]:
# Load data and features
DATA_PATH = '/home/data'

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT NAME"]]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)

print(f'Spange shape: {SPANGE_DF.shape}')
print(f'DRFP shape: {DRFP_DF.shape}')

Spange shape: (26, 13)
DRFP shape: (24, 2048)


In [3]:
# Analyze DRFP variance and select high-variance features
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
print(f'DRFP features with non-zero variance: {len(nonzero_variance_cols)}')
print(f'DRFP sparsity: {(DRFP_DF.values == 0).mean():.2%}')

# Create filtered DRFP dataframe
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]
print(f'Filtered DRFP shape: {DRFP_FILTERED.shape}')

DRFP features with non-zero variance: 122
DRFP sparsity: 97.43%
Filtered DRFP shape: (24, 122)


In [4]:
# Combined Featurizer: Spange + DRFP (filtered) + Arrhenius
class CombinedFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        # 2 numeric + 3 kinetic + 13 spange + 122 drfp = 140 features
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1]
        print(f'Total features: {self.feats_dim}')

    def featurize(self, X, flip=False):
        X_vals = X[["Residence Time", "Temperature"]].values.astype(np.float64)
        
        # Arrhenius kinetics features
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        # Solvent features
        if self.mixed:
            # Spange features
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            # DRFP features
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            
            pct = X["SolventB%"].values.reshape(-1, 1)
            
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
        
        return torch.tensor(np.hstack([X_kinetic, X_spange, X_drfp]))

# Test featurizer
test_feat = CombinedFeaturizer(mixed=False)
print(f'Feature dimension: {test_feat.feats_dim}')

Total features: 140
Feature dimension: 140


In [5]:
# MLP Model with combined features
class MLPInternal(nn.Module):
    def __init__(self, input_dim, hidden_dims=[256, 128, 64], output_dim=3, dropout=0.3):
        super(MLPInternal, self).__init__()
        layers = [nn.BatchNorm1d(input_dim)]
        
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, h_dim),
                nn.BatchNorm1d(h_dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_dim = h_dim
        
        layers.append(nn.Linear(prev_dim, output_dim))
        layers.append(nn.Sigmoid())
        
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

class CombinedMLP(nn.Module):
    def __init__(self, data='single', n_models=5):
        super().__init__()
        self.data_type = data
        self.n_models = n_models
        self.featurizer = CombinedFeaturizer(mixed=(data=='full'))
        self.models = nn.ModuleList()

    def train_model(self, X_train, y_train, epochs=300, batch_size=32, lr=5e-4):
        # Create features
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = torch.tensor(y_train.values)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = torch.cat([X_std, X_flip], dim=0)
            y_all = torch.cat([y_vals, y_vals], dim=0)
        else:
            X_all = X_std
            y_all = y_vals
            
        input_dim = X_all.shape[1]
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        for i in range(self.n_models):
            torch.manual_seed(42 + i)
            
            model = MLPInternal(input_dim, hidden_dims=[256, 128, 64], dropout=0.3).to(device)
            model.train()
            self.models.append(model)
            
            dataset = TensorDataset(X_all, y_all)
            loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            
            # Increased weight decay for regularization
            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
            criterion = nn.HuberLoss()
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode='min', factor=0.5, patience=20
            )
            
            for epoch in range(epochs):
                epoch_loss = 0.0
                for inputs, targets in loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    optimizer.step()
                    epoch_loss += loss.item() * inputs.size(0)
                
                scheduler.step(epoch_loss / len(dataset))

    def predict(self, X):
        device = next(self.models[0].parameters()).device
        
        if self.data_type == 'full':
            X_std = self.featurizer.featurize(X, flip=False).to(device)
            X_flip = self.featurizer.featurize(X, flip=True).to(device)
            
            pred_sum = torch.zeros((len(X), 3)).to(device)
            
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    p1 = model(X_std)
                    p2 = model(X_flip)
                    pred_sum += (p1 + p2) * 0.5
            
            avg_pred = pred_sum / self.n_models
        else:
            X_std = self.featurizer.featurize(X).to(device)
            pred_sum = torch.zeros((len(X), 3)).to(device)
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    pred_sum += model(X_std)
            avg_pred = pred_sum / self.n_models

        return avg_pred.cpu()

print('CombinedMLP model defined')

CombinedMLP model defined


In [6]:
# Run cross-validation for single solvent data
print("\n--- TASK 0: Single Solvent (24 folds) ---")
X, Y = load_data("single_solvent")
split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []
all_actuals = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = CombinedMLP(data='single', n_models=5)
    model.train_model(train_X, train_Y, epochs=300)
    predictions = model.predict(test_X).numpy()
    
    for row_idx, row in enumerate(predictions):
        all_predictions.append({
            "task": 0, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]
        })
    
    all_actuals.append(test_Y.values)

submission_single = pd.DataFrame(all_predictions)

actuals_single = np.vstack(all_actuals)
preds_single = submission_single[['target_1', 'target_2', 'target_3']].values
mse_single = np.mean((actuals_single - preds_single) ** 2)
print(f'\nSingle Solvent MSE: {mse_single:.6f}')


--- TASK 0: Single Solvent (24 folds) ---


  0%|          | 0/24 [00:00<?, ?it/s]

Total features: 140


  4%|▍         | 1/24 [01:47<41:07, 107.27s/it]

Total features: 140


...

 96%|█████████▌| 23/24 [37:49<01:36, 96.05s/it]

Total features: 140


100%|██████████| 24/24 [39:27<00:00, 96.55s/it]

100%|██████████| 24/24 [39:27<00:00, 98.64s/it]


Single Solvent MSE: 0.011491


In [7]:
# Run cross-validation for full data (mixtures)
print("\n--- TASK 1: Full Data (13 folds) ---")
X, Y = load_data("full")
split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions_full = []
all_actuals_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = CombinedMLP(data='full', n_models=5)
    model.train_model(train_X, train_Y, epochs=300)
    predictions = model.predict(test_X).numpy()
    
    for row_idx, row in enumerate(predictions):
        all_predictions_full.append({
            "task": 1, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]
        })
    
    all_actuals_full.append(test_Y.values)

submission_full = pd.DataFrame(all_predictions_full)

actuals_full = np.vstack(all_actuals_full)
preds_full = submission_full[['target_1', 'target_2', 'target_3']].values
mse_full = np.mean((actuals_full - preds_full) ** 2)
print(f'\nFull Data MSE: {mse_full:.6f}')


--- TASK 1: Full Data (13 folds) ---


  0%|          | 0/13 [00:00<?, ?it/s]

Total features: 140


  8%|▊         | 1/13 [05:40<1:08:11, 340.97s/it]

Total features: 140


...

 92%|█████████▏| 12/13 [1:23:16<06:55, 415.79s/it]

Total features: 140


100%|██████████| 13/13 [1:29:13<00:00, 397.96s/it]

100%|██████████| 13/13 [1:29:13<00:00, 411.78s/it]


Full Data MSE: 0.009972


In [8]:
# Calculate overall MSE
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== FINAL RESULTS ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nComparison to baselines:')
print(f'  Spange-only MLP CV: 0.011081')
print(f'  DRFP-only MLP CV: 0.016948')
print(f'  Combined (this): {overall_mse:.6f}')


=== FINAL RESULTS ===
Single Solvent MSE: 0.011491 (n=656)
Full Data MSE: 0.009972 (n=1227)
Overall MSE: 0.010501

Comparison to baselines:
  Spange-only MLP CV: 0.011081
  DRFP-only MLP CV: 0.016948
  Combined (this): 0.010501


In [9]:
# Create submission file
submission = pd.concat([submission_single, submission_full])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)
print(f'Submission saved to /home/submission/submission.csv')
print(f'Submission shape: {submission.shape}')
print(submission.head())

Submission saved to /home/submission/submission.csv
Submission shape: (1883, 7)
    index  task  fold  row  target_1  target_2  target_3
id                                                      
0       0     0     0    0  0.015884  0.016061  0.936093
1       1     0     0    1  0.022483  0.022492  0.912775
2       2     0     0    2  0.032404  0.030681  0.867469
3       3     0     0    3  0.046167  0.041265  0.806856
4       4     0     0    4  0.065455  0.055683  0.731669
